In [1]:
import pandas as pd

In [2]:
import mido
import numpy as np

def midi_to_fwod(file_path):
    """
    Takes a MIDI file path as input and returns a list of 1x16 normalized FWOD vectors.
    
    FWOD (Flattened Weighted Onset Density) is a representation where:
    - Each drum hit is weighted based on its category (low, mid, high)
    - Weights are multiplied by velocity
    - The resulting values are normalized
    
    Args:
        file_path (str): Path to the MIDI file
        
    Returns:
        list: List of 1x16 numpy arrays, each representing a flattened pattern
    """
    # MIDI General MIDI drum mapping dictionary
    GM_dict = {
        # key is midi note number
        # values are:
        # [0] name (as string)
        # [1] name category low mid or high (as string)
        # [2] substiture midi number for simplified MIDI (all instruments)
        22: ["Closed Hi-Hat edge", "high", 42],
        26: ["Open Hi-Hat edge", "high", 46],
        35: ["Acoustic Bass Drum", "low", 36],
        36: ["Bass Drum 1", "low", 36],
        37: ["Side Stick", "mid", 37],
        38: ["Acoustic Snare", "mid", 38],
        39: ["Hand Clap", "mid", 39],
        40: ["Electric Snare", "mid", 38],
        41: ["Low Floor Tom", "low", 45],
        42: ["Closed Hi Hat", "high", 42],
        43: ["High Floor Tom", "mid", 45],
        44: ["Pedal Hi-Hat", "high", 46],
        45: ["Low Tom", "low", 45],
        46: ["Open Hi-Hat", "high", 46],
        47: ["Low-Mid Tom", "low", 47],
        48: ["Hi-Mid Tom", "mid", 47],
        49: ["Crash Cymbal 1", "high", 49],
        50: ["High Tom", "mid", 50],
        51: ["Ride Cymbal 1", "high", 51],
        52: ["Chinese Cymbal", "high", 52],
        53: ["Ride Bell", "high", 53],
        54: ["Tambourine", "high", 54],
        55: ["Splash Cymbal", "high", 55],
        56: ["Cowbell", "high", 56],
        57: ["Crash Cymbal 2", "high", 57],
        58: ["Vibraslap", "mid", 58],
        59: ["Ride Cymbal 2", "high", 59],
        60: ["Hi Bongo", "high", 60],
        61: ["Low Bongo", "mid", 61],
        62: ["Mute Hi Conga", "mid", 62],
        63: ["Open Hi Conga", "high", 63],
        64: ["Low Conga", "low", 64],
        65: ["High Timbale", "mid", 65],
        66: ["Low Timbale", "low", 66],
        67: ["High Agogo", "high", 67],
        68: ["Low Agogo", "mid", 68],
        69: ["Cabasa", "high", 69],
        70: ["Maracas", "high", 69],
        71: ["Short Whistle", "high", 71],
        72: ["Long Whistle", "high", 72],
        73: ["Short Guiro", "high", 73],
        74: ["Long Guiro", "high", 74],
        75: ["Claves", "high", 75],
        76: ["Hi Wood Block", "high", 76],
        77: ["Low Wood Block", "mid", 77],
        78: ["Mute Cuica", "high", 78],
        79: ["Open Cuica", "high", 79],
        80: ["Mute Triangle", "high", 80],
        81: ["Open Triangle", "high", 81],
    }
    
    # Lists of instruments by category
    lows = [35, 36, 41, 45, 47, 64, 66]
    mids = [37, 38, 39, 40, 43, 48, 50, 61, 62, 65, 68, 77]
    highs = [22, 26, 42, 44, 46, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 63, 67, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81]
    
    # Extract patterns from MIDI file
    def midifile2hv_list(file_name):
        '''
        Get a MIDI file and convert it to a list of hv_lists (note numbers and velocity)
        Each hv_list represents a 16-step pattern
        '''
        pattern = []
        mid = mido.MidiFile(file_name)  # create a mido file instance
        sixteenth = mid.ticks_per_beat / 4  # find the length of a sixteenth note
        
        acc = 0  # use this to keep track of time
        
        for i, track in enumerate(mid.tracks):
            for msg in track:  # process all messages
                acc += msg.time  # accumulate time of any message type
                if msg.type == "note_on" and msg.velocity != 0:  # skip velocity 0 format of note off
                    if msg.note in list(GM_dict.keys()):
                        midinote = GM_dict[msg.note][2]  # remap msg.note
                        rounded_step = int((acc / sixteenth) + 0.45)
                        midivelocity = msg.velocity / 127  # normalize upfront
                        pattern.append((int(acc / sixteenth), midinote, midivelocity))  # step, note, velocity
        
        if len(pattern) > 0:  # just proceed if analyzed pattern has at least one onset
            # round the pattern to the next multiple of 16
            rounded_step = max([p[0] for p in pattern]) if pattern else 0
            if (rounded_step / 16) - (rounded_step // 16) != 0:
                pattern_len_in_steps = (rounded_step // 16) * 16 + 16
            else:
                pattern_len_in_steps = (rounded_step // 16) * 16
            
            # create an empty list of lists the size of the pattern
            output_pattern = [[] for _ in range(pattern_len_in_steps)]
            
            # group the instruments and their velocity that played at a specific step
            for step, note, velocity in pattern:
                if step < pattern_len_in_steps:
                    output_pattern[step].append((note, velocity))
                    # make sure no notes are repeated and events are sorted
                    output_pattern[step] = list(set(output_pattern[step]))
                    output_pattern[step].sort()
            
            # split the pattern every 16 steps
            hv_lists_split = []
            for x in range(len(output_pattern) // 16):
                patt_fragment = output_pattern[x * 16 : (x * 16) + 16]
                patt_density = sum([1 for x in patt_fragment if x != []])
                
                # filter out patterns that have less than 4 events with notes
                if patt_density > 4:
                    hv_lists_split.append(patt_fragment)
            
            return hv_lists_split
        return []
    
    # Flatten hv_list to FWOD representation
    def flatten_hv_list(hv_list):
        '''
        Input an hv_list and output a flattened representation as a vector
        '''
        flat = np.zeros([len(hv_list), 1])
        
        # multiply velocities and categories
        for i, step in enumerate(hv_list):
            step_weight = 0
            for onset in step:
                if onset[0] in lows:
                    step_weight += onset[1] * 3  # Low sounds weighted by 3
                elif onset[0] in mids:
                    step_weight += onset[1] * 2  # Mid sounds weighted by 2
                else:
                    step_weight += onset[1] * 1  # High sounds weighted by 1
            flat[i] = step_weight
        
        # Normalize
        max_val = max(flat) if max(flat) > 0 else 1
        flat = flat / max_val
        
        return flat
    
    # Process the MIDI file
    try:
        # Get list of 16-step patterns
        hv_lists = midifile2hv_list(file_path)
        
        # Convert each pattern to FWOD representation
        fwod_list = []
        for pattern in hv_lists:
            fwod = flatten_hv_list(pattern)
            fwod_list.append(fwod)
        
        return fwod_list
    
    except Exception as e:
        print(f"Error processing MIDI file: {e}")
        return []



In [30]:
source = pd.read_excel('balanced_all_data_1_bar.xlsx')
source[source['file'] == 'magenta midi/pop/105_pop_132_beat_4-4.mid']

# fwod_database.loc[fwod_database['midi_filename'] == 'drummer7/session2/105_pop_132_beat_4-4.mid']


,file,sequence,class,step_0,step_1,step_2,step_3,step_4,step_5,step_6,step_7,step_8,step_9,step_10,step_11,step_12,step_13,step_14,step_15
0,magenta midi/pop/105_pop_132_beat_4-4.mid,0,pop,0.000000,0.000000,0.000000,0.053571,0.0,0.000000,0.000000,1.000000,0.178571,0.071429,0.0,0.428571,0.000000,0.058036,0.0,0.000000
1,magenta midi/pop/105_pop_132_beat_4-4.mid,1,pop,0.897638,0.188976,0.000000,0.000000,1.0,0.133858,0.000000,1.000000,0.000000,0.212598,0.0,1.000000,0.000000,0.259843,0.0,1.000000
2,magenta midi/pop/105_pop_132_beat_4-4.mid,2,pop,0.000000,0.377953,0.000000,1.000000,0.0,0.000000,0.000000,0.157480,0.892388,0.049869,0.0,0.889764,0.000000,0.068241,0.0,0.774278
3,magenta midi/pop/105_pop_132_beat_4-4.mid,3,pop,0.000000,0.480315,0.000000,1.000000,0.0,0.393701,0.000000,0.333333,0.000000,0.482940,0.0,1.000000,0.000000,0.041995,0.0,0.703412
4,magenta midi/pop/105_pop_132_beat_4-4.mid,4,pop,0.094488,0.433071,0.000000,1.000000,0.0,0.000000,0.377953,0.299213,0.000000,0.419948,0.0,0.666667,0.314961,0.000000,0.0,0.000000
5,magenta midi/pop/105_pop_132_beat_4-4.mid,5,pop,0.648501,0.000000,0.376022,0.000000,1.0,0.419619,0.000000,0.237057,0.000000,0.564033,0.0,0.994550,0.000000,0.351499,0.0,0.648501
6,magenta midi/pop/105_pop_132_beat_4-4.mid,6,pop,0.000000,0.430446,0.000000,1.000000,0.0,0.275591,0.000000,0.923885,0.000000,0.118110,0.0,0.973753,0.000000,0.044619,0.0,0.737533
7,magenta midi/pop/105_pop_132_beat_4-4.mid,7,pop,0.000000,0.501340,0.000000,0.983914,0.0,0.431635,0.000000,0.294906,0.000000,0.546917,0.0,1.000000,0.000000,0.000000,0.0,0.624665
8,magenta midi/pop/105_pop_132_beat_4-4.mid,8,pop,0.000000,0.527778,0.000000,1.000000,0.0,0.472222,0.000000,0.341667,0.000000,0.405556,0.0,0.933333,0.000000,0.472222,0.0,0.669444
9,magenta midi/pop/105_pop_132_beat_4-4.mid,9,pop,0.000000,0.456693,0.000000,0.989501,0.0,0.545932,0.000000,1.000000,0.000000,0.448819,0.0,0.616798,0.000000,1.000000,0.0,0.530184


- Naive intial testing

In [ ]:
# Example usage:
# root_dir = '/Users/macbookpro/Downloads/groove-aligned'
fwod_vectors = midi_to_fwod("/Users/macbookpro/Downloads/groove-aligned/drummer7/session2/105_pop_132_beat_4-4.mid")

print(f"Found {len(fwod_vectors)} patterns")
for i, vector in enumerate(fwod_vectors):
    print(f"Pattern {i+1}: {vector.flatten()}")

- Hard implementation

In [19]:

def create_fwod_database(info_clean_df, root_dir, output_csv_path, midi_to_fwod_func):
    """
    Creates a database CSV with FWOD vectors and metadata for each bar of each MIDI file.
    
    Args:
        info_clean_df (pd.DataFrame): DataFrame containing metadata for MIDI files
                                      Must include columns: 'drummer', 'session', 'id', 'style', 
                                      'midi_filename', 'audio_filename', 'bpm', 'split'
        root_dir (str): Root directory where MIDI files are stored
        output_csv_path (str): Path where the output CSV will be saved
        midi_to_fwod_func (function): Function that converts MIDI file to FWOD vectors
        
    Returns:
        pd.DataFrame: The created DataFrame that was saved to CSV
    """
    # Create an empty list to store all rows
    all_rows = []
    
    # Process each MIDI file in the info_clean_df
    total_files = len(info_clean_df)
    
    print(f"Processing {total_files} MIDI files...")
    
    for i in range(total_files):
        # Get metadata for the current MIDI file
        row_data = info_clean_df.iloc[i]
        
        # Construct the full path to the MIDI file
        midi_path = os.path.join(root_dir, row_data['midi_filename'])
        
        try:
            # Convert MIDI to FWOD vectors
            fwod_vectors = midi_to_fwod_func(midi_path)
            
            # If we got vectors, create a row for each vector (bar)
            if fwod_vectors and len(fwod_vectors) > 0:
                for bar_number, fwod_vector in enumerate(fwod_vectors):
                    # Create a dictionary for this row
                    row_dict = {
                        'sequence': bar_number,
                        'drummer': row_data['drummer'],
                        'session': row_data['session'],
                        'id': row_data['id'],
                        'style': row_data['style'],
                        'midi_filename': row_data['midi_filename'],
                        'audio_filename': row_data['audio_filename'],
                        'bpm': row_data['bpm'],
                        'split': row_data['split']
                    }
                    
                    # Add the 16 FWOD values (y_0 to y_15)
                    for step in range(16):
                        row_dict[f'y_{step}'] = fwod_vector[step][0]
                    
                    # Add this row to our collection
                    all_rows.append(row_dict)
                
                if (i + 1) % 10 == 0 or (i + 1) == total_files:
                    print(f"Processed {i+1}/{total_files} files, extracted {len(all_rows)} bars so far...")
                
            else:
                print(f"Warning: No FWOD vectors extracted from {midi_path}")
                
        except Exception as e:
            print(f"Error processing {midi_path}: {e}")
    
    # Create DataFrame from all rows
    fwod_df = pd.DataFrame(all_rows)
    
    # Save to CSV
    fwod_df.to_csv(output_csv_path, index=False)
    print(f"Database saved to {output_csv_path} with {len(fwod_df)} total bars")
    
    return fwod_df

In [21]:
import os
# Example usage:
fwod_database = create_fwod_database(
    info_clean_df=info_clean, 
    root_dir='/Users/macbookpro/Downloads/groove-aligned',
    output_csv_path='fwod_database.csv',
    midi_to_fwod_func=midi_to_fwod
)

Processing 1090 MIDI files...
Processed 10/1090 files, extracted 160 bars so far...
Processed 20/1090 files, extracted 440 bars so far...
Processed 30/1090 files, extracted 450 bars so far...
Processed 40/1090 files, extracted 460 bars so far...
Processed 50/1090 files, extracted 541 bars so far...
Processed 60/1090 files, extracted 552 bars so far...
Processed 70/1090 files, extracted 657 bars so far...
Processed 80/1090 files, extracted 668 bars so far...
Processed 90/1090 files, extracted 679 bars so far...
Processed 100/1090 files, extracted 787 bars so far...
Processed 110/1090 files, extracted 889 bars so far...
Processed 120/1090 files, extracted 899 bars so far...
Processed 130/1090 files, extracted 1069 bars so far...
Processed 140/1090 files, extracted 1079 bars so far...
Processed 150/1090 files, extracted 1090 bars so far...
Processed 160/1090 files, extracted 1105 bars so far...
Processed 170/1090 files, extracted 1220 bars so far...
Processed 180/1090 files, extracted 123

- Small review

In [29]:
fwod_database = pd.read_csv('fwod_database.csv')
# /Users/macbookpro/Downloads/groove-aligned/drummer7/session2/105_pop_132_beat_4-4.mid
fwod_database.loc[fwod_database['midi_filename'] == 'drummer7/session2/105_pop_132_beat_4-4.mid']

,sequence,drummer,session,id,style,midi_filename,audio_filename,bpm,split,y_0,...,y_6,y_7,y_8,y_9,y_10,y_11,y_12,y_13,y_14,y_15
13236,0,drummer7,drummer7/session2,drummer7/session2/105,pop,drummer7/session2/105_pop_132_beat_4-4.mid,drummer7/session2/105_pop_132_beat_4-4.wav,132,train,0.000000,...,0.000000,1.000000,0.178571,0.071429,0.0,0.428571,0.000000,0.058036,0.0,0.000000
13237,1,drummer7,drummer7/session2,drummer7/session2/105,pop,drummer7/session2/105_pop_132_beat_4-4.mid,drummer7/session2/105_pop_132_beat_4-4.wav,132,train,0.897638,...,0.000000,1.000000,0.000000,0.212598,0.0,1.000000,0.000000,0.259843,0.0,1.000000
13238,2,drummer7,drummer7/session2,drummer7/session2/105,pop,drummer7/session2/105_pop_132_beat_4-4.mid,drummer7/session2/105_pop_132_beat_4-4.wav,132,train,0.000000,...,0.000000,0.157480,0.892388,0.049869,0.0,0.889764,0.000000,0.068241,0.0,0.774278
13239,3,drummer7,drummer7/session2,drummer7/session2/105,pop,drummer7/session2/105_pop_132_beat_4-4.mid,drummer7/session2/105_pop_132_beat_4-4.wav,132,train,0.000000,...,0.000000,0.333333,0.000000,0.482940,0.0,1.000000,0.000000,0.041995,0.0,0.703412
13240,4,drummer7,drummer7/session2,drummer7/session2/105,pop,drummer7/session2/105_pop_132_beat_4-4.mid,drummer7/session2/105_pop_132_beat_4-4.wav,132,train,0.094488,...,0.377953,0.299213,0.000000,0.419948,0.0,0.666667,0.314961,0.000000,0.0,0.000000
13241,5,drummer7,drummer7/session2,drummer7/session2/105,pop,drummer7/session2/105_pop_132_beat_4-4.mid,drummer7/session2/105_pop_132_beat_4-4.wav,132,train,0.648501,...,0.000000,0.237057,0.000000,0.564033,0.0,0.994550,0.000000,0.351499,0.0,0.648501
13242,6,drummer7,drummer7/session2,drummer7/session2/105,pop,drummer7/session2/105_pop_132_beat_4-4.mid,drummer7/session2/105_pop_132_beat_4-4.wav,132,train,0.000000,...,0.000000,0.923885,0.000000,0.118110,0.0,0.973753,0.000000,0.044619,0.0,0.737533
13243,7,drummer7,drummer7/session2,drummer7/session2/105,pop,drummer7/session2/105_pop_132_beat_4-4.mid,drummer7/session2/105_pop_132_beat_4-4.wav,132,train,0.000000,...,0.000000,0.294906,0.000000,0.546917,0.0,1.000000,0.000000,0.000000,0.0,0.624665
13244,8,drummer7,drummer7/session2,drummer7/session2/105,pop,drummer7/session2/105_pop_132_beat_4-4.mid,drummer7/session2/105_pop_132_beat_4-4.wav,132,train,0.000000,...,0.000000,0.341667,0.000000,0.405556,0.0,0.933333,0.000000,0.472222,0.0,0.669444
13245,9,drummer7,drummer7/session2,drummer7/session2/105,pop,drummer7/session2/105_pop_132_beat_4-4.mid,drummer7/session2/105_pop_132_beat_4-4.wav,132,train,0.000000,...,0.000000,1.000000,0.000000,0.448819,0.0,0.616798,0.000000,1.000000,0.0,0.530184
